In [3]:

import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.themoviedb.org'
header={'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
mylst = []

for items in range(1,26):
        # using loop to generate urls
        number = str(items)
        response = requests.get(url+'/movie?page='+number, headers = header).text
        soupdata = BeautifulSoup(response)
        divs = soupdata.find_all('div', class_='card style_1')

        for items in divs:
                # using for loop to remove html code of all divs from a single page
                name = items.find('h2')  # extracting movie name
                link = name.find('a')['href']
                movie_name = items.find('h2').text  

                ans_url = url + link  # creating url which appears when we click on a movie                        

                inside_page_code = requests.get(ans_url,headers = header).text  # Extracting Runtime 
                soupdata_of_inpage = BeautifulSoup(inside_page_code,'lxml')  # Using conditional statements to extract 
                                                                             # the runtime
                if soupdata_of_inpage.find('span',class_='runtime') == None:
                    runtime = 'No data available'

                elif soupdata_of_inpage.find('span',class_='runtime') != None:
                    runtime = soupdata_of_inpage.find('span',class_='runtime').text

                rating = soupdata_of_inpage.find('div',class_='percent')  # Extracting Rating of a movie 
                                                                          # using Regular Expression
                c = rating.find('span')['class'][1]
                pattern = '\d+'
                final_rating = re.findall(pattern,c)
                Rating = ''
                for x in final_rating:
                    Rating = int(x)

                release_date=soupdata_of_inpage.find('span',class_='release').text # Extracting release date 
                                                                                   # using Regular expression       
                pattern1 = '\d+[/]\d+[/]\d+'
                final_release_date = re.findall(pattern1,release_date)
                for items in final_release_date:
                    Release_Date=items

                genres=soupdata_of_inpage.find('span',class_='genres').text  # Extracting genres for a movie
                genres=genres.strip()

                direct_name = soupdata_of_inpage.find('ol',class_='people no_image')
                a = direct_name.find_all('li',class_='profile')  # Extracting Director Name using IF condition
                for items in a:
                    b=items.find('p',class_='character').text
                    if b.count('Director') == 1:
                        director_name = items.find('a').text

                mydict = {}  # Creating a dictionary using all the above data
                mydict['Name'] = movie_name
                mydict['Rating'] = Rating
                mydict['Genre'] = genres
                mydict['Release date'] = Release_Date
                mydict['Runtime'] = runtime
                mydict['Director'] = director_name
                mydict['Url'] = ans_url

                mylst.append(mydict)  # Appending the dictionary in a list
        
import pandas  as pd  # using pandas library to convert the data stored in a list into a tabular format
l = pd.DataFrame(mylst)

l.to_excel('movies_info.xlsx')  # using the above table and converting the data into a excelsheet
